In [ ]:
#  Graphs
#  Nodes: Agents or functions
#  Edges: connect nodes
#  Conditional edges: decisions

In [22]:
# LangGraph Components - Google Gemini API Version (Function Call Fix)
from dotenv import load_dotenv
import os
_ = load_dotenv()

import google.generativeai as genai
from google.generativeai.types import FunctionDeclaration, Tool
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
import re
import logging

In [23]:
# Configure logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# Configure Google Gemini API
genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))
print("✅ Google Gemini API configured!")

# Initialize DuckDuckGo search
search_wrapper = DuckDuckGoSearchAPIWrapper()

def execute_search(query: str) -> str:
    """Execute search using DuckDuckGo."""
    try:
        results = search_wrapper.run(query)
        return results[:500] if results else "No results found."
    except Exception as e:
        return f"Search error: {str(e)}"

# Gemini-native tool
search_tool = Tool(
    function_declarations=[
        FunctionDeclaration(
            name="google_search",
            description="Search Google for current information. Use this for up-to-date facts.",
            parameters={
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The search query."}
                },
                "required": ["query"]
            }
        )
    ]
)

print(f"✅ Search tool ready: google_search")

# Agent State
class AgentState(TypedDict):
    messages: Annotated[List[AnyMessage], operator.add]

✅ Google Gemini API configured!
✅ Search tool ready: google_search


In [24]:
# Gemini Agent Class
class GeminiAgent:
    def __init__(self, system=""):
        self.system = system
        self.model = genai.GenerativeModel(
            model_name='gemini-2.5-flash',
            tools=[search_tool],
            system_instruction=system
        )
        
        # Build LangGraph
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_gemini)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.should_take_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
    
    def should_take_action(self, state: AgentState) -> bool:
        """Check if the last message contains a tool call."""
        last_msg = state['messages'][-1]
        return "Action: google_search:" in str(last_msg.content)
    
    def call_gemini(self, state: AgentState):
        """Call Gemini with conversation history."""
        messages = state['messages']
        gemini_history = [msg.content for msg in messages if not isinstance(msg, SystemMessage)]
        
        try:
            logger.debug(f"Generating content with history: {gemini_history}")
            response = self.model.generate_content(gemini_history)
            logger.debug(f"Raw response: {response}")
            
            # Handle function calls from parts
            if hasattr(response, 'candidates') and response.candidates:
                candidate = response.candidates[0]
                if hasattr(candidate, 'content') and candidate.content.parts:
                    for part in candidate.content.parts:
                        if hasattr(part, 'function_call'):
                            logger.debug(f"Function call detected: {part.function_call}")
                            fc = part.function_call
                            if fc.name == "google_search":
                                query = fc.args.get('query', '')
                                return {'messages': [HumanMessage(
                                    content=f"Thought: Need to search for {query}.\nAction: google_search: {query}"
                                )]}
            
            # Handle text-based response if no function call
            if hasattr(response, 'text') and response.text:
                logger.debug(f"Text response: {response.text}")
                ai_message = HumanMessage(content=response.text)
                if "?" in response.text or "search" in response.text.lower():
                    logger.debug("Triggering search due to question or search keyword")
                    ai_message.content = f"Thought: I need to search.\nAction: google_search: {messages[-1].content}"
            else:
                logger.debug("No text or function call in response")
                ai_message = HumanMessage(content="No actionable response from Gemini.")
        except Exception as e:
            logger.error(f"Exception in call_gemini: {str(e)}")
            ai_message = HumanMessage(content=f"Error: {str(e)}")
        
        return {'messages': [ai_message]}
    
    def take_action(self, state: AgentState):
        """Execute tool calls."""
        last_msg = state['messages'][-1].content
        
        action_match = re.search(r'Action:\s*(\w+):\s*(.*)', last_msg, re.MULTILINE)
        if not action_match:
            logger.debug("No action match found in message")
            return {'messages': [ToolMessage(content="No action found.", tool_call_id="0", name="none")]}
        
        action_name, query = action_match.groups()
        logger.debug(f"Executing action: {action_name} with query: {query}")
        print(f"🔍 Calling: {action_name}('{query}')")
        
        if action_name == "google_search":
            result = execute_search(query)
        else:
            result = f"❌ Unknown tool: {action_name}"
        
        tool_msg = ToolMessage(content=result, tool_call_id="1", name=action_name)
        print("✅ Back to Gemini!")
        return {'messages': [tool_msg]}
  

In [25]:
# Initialize Agent
prompt = """You are a smart research assistant. Use google_search for facts.
Format: Thought: [reasoning]\nAction: google_search: [query]\nAnswer: [final]"""
abot = GeminiAgent(system=prompt)
print("✅ Gemini Agent ready!")

# Test Super Bowl Query
query = """Who won the super bowl in 2024? In what state is the winning team headquarters located? 
What is the GDP of that state? Answer each question."""
messages = [HumanMessage(content=query)]
result = abot.graph.invoke({"messages": messages})
print("\n🏈 SUPER BOWL 2024 RESULTS:")
print("=" * 50)
print(result['messages'][-1].content)

DEBUG:__main__:Generating content with history: ['Who won the super bowl in 2024? In what state is the winning team headquarters located? \nWhat is the GDP of that state? Answer each question.']


✅ Gemini Agent ready!


DEBUG:__main__:Raw response: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "google_search",
                  "args": {
                    "query": "who won super bowl 2024"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 127,
        "candidates_token_count": 23,
        "total_token_count": 261
      },
      "model_version": "gemini-2.5-flash"
    }),
)
DEBUG:__main__:Function call detected: name: "google_search"
args {
  fields {
    key: "query"
    value {
      string_value: "who won super bowl 2024"
    }
  }
}

DEBUG:__main__:Executing action: google_search with query:

🔍 Calling: google_search('who won super bowl 2024')


DEBUG:primp.utils:Loaded CA certs
DEBUG:primp.utils:Loaded CA certs
DEBUG:primp.utils:Loaded CA certs
DEBUG:primp.utils:Loaded CA certs
DEBUG:primp.utils:Loaded CA certs
DEBUG:primp.utils:Loaded CA certs
DEBUG:rquest.util.client.connect.dns:resolving wt.wikipedia.org
DEBUG:rquest.util.client.connect.dns:resolving search.yahoo.com
INFO:ddgs.ddgs:Error in engine wikipedia: DDGSException("RuntimeError: RuntimeError('error sending request for url (https://wt.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=who%20won%20super%20bowl%202024): client error (Connect)\\n\\nCaused by:\\n    0: client error (Connect)\\n    1: dns error: No such host is known. (os error 11001)\\n    2: No such host is known. (os error 11001)')")
DEBUG:__main__:Generating content with history: ['Who won the super bowl in 2024? In what state is the winning team headquarters located? \nWhat is the GDP of that state? Answer each question.', 'Thought: Need to search for who won super bowl 2024.\nAc

...

🔍 Calling: google_search('Missouri GDP 2024')


DEBUG:primp.utils:Loaded CA certs
DEBUG:primp.utils:Loaded CA certs
DEBUG:primp.utils:Loaded CA certs
DEBUG:rquest.util.client.connect.dns:resolving wt.wikipedia.org
DEBUG:rquest.util.client.connect.dns:resolving search.brave.com
INFO:ddgs.ddgs:Error in engine wikipedia: DDGSException("RuntimeError: RuntimeError('error sending request for url (https://wt.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=Missouri%20GDP%202024): client error (Connect)\\n\\nCaused by:\\n    0: client error (Connect)\\n    1: dns error: No such host is known. (os error 11001)\\n    2: No such host is known. (os error 11001)')")
DEBUG:rquest.util.client.connect.dns:resolving www.bing.com
DEBUG:__main__:Generating content with history: ['Who won the super bowl in 2024? In what state is the winning team headquarters located? \nWhat is the GDP of that state? Answer each question.', 'Thought: Need to search for who won super bowl 2024.\nAction: google_search: who won super bowl 2024', 'The 

✅ Back to Gemini!


DEBUG:__main__:Raw response: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The Kansas City Chiefs won Super Bowl LVIII in 2024.\nThe winning team's headquarters are located in Missouri.\nMissouri's GDP in 2024 was $359.952 billion (real GDP) and it was the 22nd largest state economy in the nation in 2021."
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 2271,
        "candidates_token_count": 74,
        "total_token_count": 2459,
        "cached_content_token_count": 1730
      },
      "model_version": "gemini-2.5-flash"
    }),
)
DEBUG:__main__:Function call detected: 
DEBUG:__main__:Text response: The Kansas City Chiefs won Super Bowl LVIII in 202


🏈 SUPER BOWL 2024 RESULTS:
The Kansas City Chiefs won Super Bowl LVIII in 2024.
The winning team's headquarters are located in Missouri.
Missouri's GDP in 2024 was $359.952 billion (real GDP) and it was the 22nd largest state economy in the nation in 2021.
